In [1]:
import torch

# torch.get_default_device() 함수가 없다고 떠서 대체 함수 정의


def _get_default_device():
    """torch.get_default_device() 대체 함수"""

    # gpu가 있으면 cuda 디바이스로 반환
    if torch.cuda.is_available():
        return torch.device("cuda")
    # apple m1,m2 칩이 있으면 mps 디바이스로 반환
    elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        return torch.device("mps")
    # 그 외는 cpu 디바이스로 반환
    else:
        return torch.device("cpu")

## 라이브러리 임포트

In [2]:
import os
import torch
import pickle
import copy
from dotenv import load_dotenv
from langchain_core.prompts import load_prompt
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_teddynote import logging
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
import torch.nn.functional as F
import re
from soynlp.normalizer import repeat_normalize
import pandas as pd
import os
import pickle
import copy
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import torch.nn.functional as F
import re
from soynlp.normalizer import repeat_normalize
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from langchain_core.prompts import load_prompt
import datetime
import pandas as pd
from geopy.distance import geodesic
from dataclasses import dataclass
from typing import Tuple, Optional, Dict, List

load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")
# 클로드 api 키를 환경 변수에서 불러옴
anthropic_key = os.getenv("ANTHROPIC_API_KEY")

# 제미나이 api 키를 환경 변수에서 불러옴
api_key3 = os.getenv("GOOGLE_API_KEY")

# 프로젝트 이름을 입력합니다.
logging.langsmith("chatnge_ai")

llm = ChatOpenAI()

LangSmith 추적을 시작합니다.
[프로젝트명]
chatnge_ai


In [3]:
load_dotenv()

True

In [4]:
# transformer 임포트 전에 환경 설정

# transformers 라이브러리가 온라인 모드로 동작하도록 설정
## 모델 다운로드 설정
os.environ["TRANSFORMERS_OFFLINE"] = "0"
# KLUE/BERT 모델 사용하기 위해 Hugging Face 모델 접근할 수 있도록 설정
os.environ["HF_HUB_OFFLINE"] = "0"

In [5]:
try:
    from transformers import AutoTokenizer, AutoModel, logging as transformers_logging

    transformers_logging.set_verbosity_error()  # 경고 메시지 최소화
    print("✅ Transformers 라이브러리 로드 완료")
except ImportError as e:
    print(f"❌ Transformers 라이브러리 임포트 실패: {e}")
    print("pip install transformers를 실행해주세요.")
    raise

✅ Transformers 라이브러리 로드 완료


In [6]:
## torch에 없는 함수 추가
torch.get_default_device = _get_default_device

## 데이터 전처리

In [7]:
# 한국어 불용어 리스트
KOREAN_STOPWORDS = [
    "이",
    "그",
    "저",
    "것",
    "및",
    "에",
    "를",
    "은",
    "는",
    "이런",
    "저런",
    "그런",
    "한",
    "이르",
    "또한",
    "있",
    "하",
    "에서",
    "으로",
    "으로써",
    "로써",
    "로서",
    "로",
    "와",
    "과",
    "이고",
    "이며",
    "이다",
    "있다",
    "하다",
    "되다",
    "이",
    "가",
    "을",
    "를",
    "에게",
    "의",
    "뿐",
    "다",
    "적",
    "데",
    "때",
    "나",
    "도",
    "만",
    "께",
    "에게서",
]

In [8]:
# 한국어 텍스트 전처리
def preprocess_korean_text(text):
    # 만약에 텍스트가 없으면 그냥 빈칸 반환
    if pd.isna(text) or text is None or len(str(text).strip()) == 0:
        return ""

    text = str(text)
    text = re.sub(r"(.)\1{2,}", r"\1\1", text)  # 반복 문자 정규화
    text = re.sub(r"[^가-힣a-zA-Z0-9\s\.,!?]", " ", text)  # 특수문자 제거
    text = re.sub(r"\s+", " ", text).strip()  # 공백 정리

    return text


def remove_stopwords(text, stopwords=KOREAN_STOPWORDS):
    """주어진 텍스트에서 불용어 제거"""
    # 아무것도 없으면 그냥 반환
    if pd.isna(text) or text is None or len(str(text).strip()) == 0:
        return ""

    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]

    return " ".join(filtered_words)

## 감정 분류기 모델

In [9]:
class EmotionClassifier(torch.nn.Module):
    """감정 분류기 모델 - BERT 기반의 감정 분류를 위한 신경망 모델"""

    def __init__(self, bert_model, num_classes, dropout_rate=0.3):
        """
        모델 초기화

        Args:
            bert_model: 사전 훈련된 BERT 모델 (예: BertModel)
            num_classes: 분류할 감정 클래스의 개수
            dropout_rate: 드롭아웃 비율 (기본값: 0.3)
        """
        super(EmotionClassifier, self).__init__()

        # BERT 모델을 백본으로 사용
        self.bert = bert_model
        # BERT의 히든 사이즈 (일반적으로 768)
        self.hidden_size = self.bert.config.hidden_size

        # 과적합 방지를 위한 드롭아웃 레이어
        self.dropout1 = torch.nn.Dropout(dropout_rate)

        # 어텐션 메커니즘: 각 토큰의 중요도를 계산
        self.attention = torch.nn.Sequential(
            # 히든 사이즈를 256차원으로 축소
            torch.nn.Linear(self.hidden_size, 256),
            # Tanh 활성화 함수로 비선형성 추가
            torch.nn.Tanh(),
            # 256차원을 1차원으로 축소하여 어텐션 스코어 생성
            torch.nn.Linear(256, 1),
            # Softmax로 어텐션 가중치를 정규화 (합이 1이 되도록)
            torch.nn.Softmax(dim=1),
        )

        # 최종 분류를 위한 분류기
        self.classifier = torch.nn.Sequential(
            # 히든 사이즈를 256차원으로 축소
            torch.nn.Linear(self.hidden_size, 256),
            # ReLU 활성화 함수
            torch.nn.ReLU(),
            # 배치 정규화로 학습 안정성 향상
            torch.nn.BatchNorm1d(256),
            # 드롭아웃으로 과적합 방지
            torch.nn.Dropout(dropout_rate),
            # 최종 감정 클래스 개수만큼 출력
            torch.nn.Linear(256, num_classes),
        )

    def forward(self, input_ids, attention_mask, token_type_ids):
        """
        순전파 과정

        Args:
            input_ids: 토크나이징된 입력 텍스트의 ID
            attention_mask: 패딩 토큰을 무시하기 위한 마스크
            token_type_ids: 문장 구분을 위한 토큰 타입 ID

        Returns:
            logits: 각 감정 클래스에 대한 점수
        """
        # BERT 모델에 입력을 통과시켜 임베딩 생성
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=True,  # 딕셔너리 형태로 결과 반환
        )

        # 모든 토큰의 히든 상태 (배치 크기, 시퀀스 길이, 히든 크기)
        sequence_output = outputs.last_hidden_state
        # [CLS] 토큰의 풀링된 표현 (배치 크기, 히든 크기)
        pooled_output = outputs.pooler_output

        # 어텐션 가중치 계산: 각 토큰의 중요도 결정
        attention_weights = self.attention(sequence_output)

        # 가중합을 통해 컨텍스트 벡터 생성
        # attention_weights와 sequence_output을 곱하고 시퀀스 차원을 따라 합산
        context_vector = torch.sum(attention_weights * sequence_output, dim=1)

        # 어텐션 기반 컨텍스트 벡터와 BERT의 풀링 출력을 결합
        # 두 표현을 더해서 더 풍부한 표현 생성
        final_output = context_vector + pooled_output

        # 드롭아웃 적용하여 과적합 방지
        final_output = self.dropout1(final_output)

        # 분류기를 통과시켜 최종 로짓 계산
        logits = self.classifier(final_output)

        return logits

## 계층적 감정 분류기

In [10]:
class HierarchicalEmotionClassifier:
    def __init__(self, model_dir, confidence_threshold=0.6, emotion_threshold=0.3):

        self.model_dir = model_dir  # 모델 경로 설정
        self.device = _get_default_device()  # 디바이스 설정
        self.max_len = 128  # 최대 길이 설정ㅁ

        # 디버깅용
        print(f"모델 디렉토리: {self.model_dir}")
        print(f"디바이스: {self.device}")

        # 임계값 설정
        self.confidence_threshold = confidence_threshold  # 신뢰도 임계값
        self.emotion_threshold = emotion_threshold  # 감정 탐지 임계값

        print(f"신뢰도 임계값: {self.confidence_threshold}")
        print(f"감정 탐지 임계값: {self.emotion_threshold}")

        # 변수 초기화
        self.tokenizer = None
        self.bert_model = None
        self.level1_model = None
        self.level2_model = None
        self.level3_model = None
        self.level1_labels = None
        self.level2_labels = None
        self.level3_labels = None

        # 모델 로드
        self._load_models()

    def _check_files(self):
        print("1단계 : 파일 확인")
        # 필수 파일들 정의
        required_files = [
            "level1_best_model.pt",
            "level2_best_model.pt",
            "level3_best_model.pt",
            "level1_label_encoder.pkl",
            "level2_label_encoder.pkl",
            "level3_label_encoder.pkl",
        ]

        # 없는 파일 확인 리스트
        missing_files = []

        for file in required_files:
            filepath = os.path.join(self.model_dir, file)
            if not os.path.exists(filepath):
                missing_files.append(file)
            else:
                print(f"✅ {file} 파일이 존재합니다.")

        # 만약에 누락된 파일이 있으면 예외 발생
        if missing_files:
            raise FileExistsError(f"파일 누락 목록:{missing_files}")

    # 라벨 인코터 로드
    def _load_label_encoders(self):
        print("2단계: 라벨 인코더 로드")

        encoders = [
            ("level1_label_encoder.pkl", "level1_encoder"),
            ("level2_label_encoder.pkl", "level2_encoder"),
            ("level3_label_encoder.pkl", "level3_encoder"),
        ]

        for filename, attr_name in encoders:
            filepath = os.path.join(self.model_dir, filename)
            with open(filepath, "rb") as f:
                encoder = pickle.load(f)
                setattr(self, attr_name, encoder)
            print(f"{filename} 파일에서 {attr_name} 로드 완료")

    # 3단계 bert 모델 로드
    def _load_bert_model(self):

        try:
            print("3단계: BERT 모델 로드")
            # BERT 모델과 토크나이저 로드
            self.tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
            self.bert_model = AutoModel.from_pretrained("klue/bert-base")

            # 디바이스 이동
            self.bert_model = self.bert_model.to(self.device)
            print("BERT 모델과 토크나이저 로드 완료")
        except Exception as e:
            print(f"❌ BERT 모델 로드 실패: {e}")
            raise ImportError(
                "BERT 모델을 로드하는 데 실패했습니다. 'transformers' 라이브러리가 설치되어 있는지 확인하세요."
            )

    def _load_emotion_models(self):
        """감정 분류 모델들 로드"""
        print("😊 감정 분류 모델 로드 중...")

        models = [
            ("level1_best_model.pt", "level1_model", self.level1_encoder),
            ("level2_best_model.pt", "level2_model", self.level2_encoder),
            ("level3_best_model.pt", "level3_model", self.level3_encoder),
        ]

        for filename, attr_name, encoder in models:
            filepath = os.path.join(self.model_dir, filename)

            print(f"  🔄 {filename} 로드 중...")

            # 모델 초기화
            model = EmotionClassifier(
                copy.deepcopy(self.bert_model.to("cpu")), len(encoder.classes_)
            )

            # 가중치 로드
            try:
                state_dict = torch.load(filepath, map_location="cpu", weights_only=True)
            except TypeError:  # 구버전 PyTorch
                state_dict = torch.load(filepath, map_location="cpu")

            model.load_state_dict(state_dict)

            # 디바이스 이동 및 평가 모드
            model = model.to(self.device)
            model.eval()

            setattr(self, attr_name, model)
            print(f"  ✅ {filename} 로드 완료")

    def _load_models(self):
        """모든 모델 로드"""
        try:
            # 1단계 : 파일 존재 확인
            self._check_files()

            # 2단계 : 라벨 인코더 로드
            self._load_label_encoders()

            # 3단계
            self._load_bert_model()

            # 4단계
            self._load_emotion_models()

        except Exception as e:
            print(f"❌ 모델 로드 중 오류 발생: {e}")
            raise

    # 단일 모델 분류
    def _predict_single(self, model, text):
        # 전처리
        preprocessed = preprocess_korean_text(text)
        preprocessed = remove_stopwords(preprocessed)

        # 토큰화
        encoding = self.tokenizer.encode_plus(
            preprocessed,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )

        # 디바이스 이동
        input_ids = encoding["input_ids"].to(self.device)
        attention_mask = encoding["attention_mask"].to(self.device)
        token_type_ids = encoding["token_type_ids"].to(self.device)

        # 예측
        with torch.no_grad():  # 그래디어언트 계산 비활성화 : 메모리 절약 + 속도 향상
            outputs = model(input_ids, attention_mask, token_type_ids)  # 로짓 출력
            probs = torch.nn.functional.softmax(
                outputs, dim=1
            )  # 확률 변환, 각 클래스별 확률값
            _, preds = torch.max(outputs, dim=1)  # 가장 높은 확률을 가진 클래스 선택

        return preds.item(), probs[0]

    # 계층적 감정 분류
    def predict_hierarchical(self, text):
        """계층적 예측"""
        result = {
            "original_text": text,
            "preprocessed_text": remove_stopwords(preprocess_korean_text(text)),
            "levels": {},
        }

        # 1단계
        pred1, probs1 = self._predict_single(self.level1_model, text)
        label1 = self.level1_encoder.inverse_transform([pred1])[0]

        result["levels"]["level1"] = {
            "step": "1단계: 일반대화 vs 감정",
            "prediction": label1,
            "confidence": float(probs1[pred1]),
            "probabilities": {
                self.level1_encoder.classes_[i]: float(probs1[i])
                for i in range(len(self.level1_encoder.classes_))
            },
        }

        if label1 == "일반대화":
            result["final"] = {
                "prediction": "일반대화",
                "confidence": float(probs1[pred1]),
            }
            result["path"] = ["일반대화"]
            return result

        # 2단계
        pred2, probs2 = self._predict_single(self.level2_model, text)
        label2 = self.level2_encoder.inverse_transform([pred2])[0]

        result["levels"]["level2"] = {
            "step": "2단계: 기쁨 vs 기타감정",
            "prediction": label2,
            "confidence": float(probs2[pred2]),
            "probabilities": {
                self.level2_encoder.classes_[i]: float(probs2[i])
                for i in range(len(self.level2_encoder.classes_))
            },
        }

        if label2 == "기쁨":
            result["final"] = {"prediction": "기쁨", "confidence": float(probs2[pred2])}
            result["path"] = ["감정", "기쁨"]
            return result

        # 3단계
        pred3, probs3 = self._predict_single(self.level3_model, text)
        label3 = self.level3_encoder.inverse_transform([pred3])[0]

        result["levels"]["level3"] = {
            "step": "3단계: 세부 감정 분류",
            "prediction": label3,
            "confidence": float(probs3[pred3]),
            "probabilities": {
                self.level3_encoder.classes_[i]: float(probs3[i])
                for i in range(len(self.level3_encoder.classes_))
            },
        }

        result["final"] = {"prediction": label3, "confidence": float(probs3[pred3])}
        result["path"] = ["감정", "기타감정", label3]

        return result

    # 신뢰도 기반 복합 감정 분석
    def predict_with_confidence_analysis(self, text, threshold=0.55):
        result = self.predict_hierarchical(text)

        # 각 단계별 신뢰도 분석
        analysis = {
            "original_text": text,
            "complexity_level": "simple",  # simple, mixed, complex
            "primary_emotion": result["final"]["prediction"],
            "primary_confidence": result["final"]["confidence"],
            "mixed_emotions": [],
            "uncertainty_indicators": [],
        }

        # Level 2에서 기쁨과 기타감정 혼동 체크
        if "level2" in result["levels"]:
            level2_probs = result["levels"]["level2"]["probabilities"]
            joy_prob = level2_probs.get("기쁨", 0)
            other_prob = level2_probs.get("기타감정", 0)

            # 확률 차이가 작으면 혼합 감정으로 판단
            prob_diff = abs(joy_prob - other_prob)

            if prob_diff < 0.3:  # 30% 이하 차이면 혼합 감정
                analysis["complexity_level"] = "mixed"
                analysis["mixed_emotions"].append(
                    {"emotion": "기쁨", "confidence": joy_prob, "type": "positive"}
                )

                # Level 3에서 세부 감정도 추가
                if "level3" in result["levels"]:
                    level3_result = result["levels"]["level3"]
                    analysis["mixed_emotions"].append(
                        {
                            "emotion": level3_result["prediction"],
                            "confidence": level3_result["confidence"],
                            "type": "negative_or_neutral",
                        }
                    )
        if threshold is None:
            threshold = self.confidence_threshold

        # 전반적 신뢰도가 낮으면 복잡한 감정으로 분류
        if result["final"]["confidence"] < threshold:
            analysis["complexity_level"] = "complex"
            analysis["uncertainty_indicators"].append(
                f"낮은 신뢰도: {result['final']['confidence']:.3f}"
            )

        return analysis

    def predict_multi_emotion_threshold(self, text, threshold=None):
        """임계값 이상의 모든 감정 반환 (완성된 버전)"""
        # 임계값이 지정되지 않으면 클래스 기본값 사용
        if threshold is None:
            threshold = self.emotion_threshold

        result = {
            "original_text": text,
            "preprocessed_text": remove_stopwords(preprocess_korean_text(text)),
            "detected_emotions": [],
            "emotion_combination": None,
        }

        # 모든 단계에서 임계값 이상인 감정들 수집
        hierarchical_result = self.predict_hierarchical(text)

        all_emotions = []

        # Level 1 체크
        if "level1" in hierarchical_result["levels"]:
            for emotion, prob in hierarchical_result["levels"]["level1"][
                "probabilities"
            ].items():
                if prob >= threshold:
                    all_emotions.append(
                        {
                            "emotion": emotion,
                            "probability": prob,
                            "level": 1,
                            "category": "conversation_type",
                        }
                    )

        # Level 2 체크 (기쁨 vs 기타감정)
        if "level2" in hierarchical_result["levels"]:
            for emotion, prob in hierarchical_result["levels"]["level2"][
                "probabilities"
            ].items():
                if prob >= threshold:
                    all_emotions.append(
                        {
                            "emotion": emotion,
                            "probability": prob,
                            "level": 2,
                            "category": "emotion_polarity",
                        }
                    )

        # Level 3 체크 (세부 감정)
        if "level3" in hierarchical_result["levels"]:
            for emotion, prob in hierarchical_result["levels"]["level3"][
                "probabilities"
            ].items():
                if prob >= threshold:
                    all_emotions.append(
                        {
                            "emotion": emotion,
                            "probability": prob,
                            "level": 3,
                            "category": "specific_emotion",
                        }
                    )

        result["detected_emotions"] = sorted(
            all_emotions, key=lambda x: x["probability"], reverse=True
        )

        # 감정 조합 분석
        result["emotion_combination"] = self._analyze_emotion_combination(all_emotions)

        return result

    def _analyze_emotion_combination(self, emotions):
        """감정 조합 분석"""
        # Level 2에서 기쁨과 기타감정이 모두 임계값 이상인지 체크
        level2_emotions = [e for e in emotions if e["level"] == 2]
        level3_emotions = [e for e in emotions if e["level"] == 3]

        combination_type = "single"
        details = {}

        if len(level2_emotions) >= 2:  # 기쁨과 기타감정 둘 다 높은 확률
            joy_emotion = next(
                (e for e in level2_emotions if e["emotion"] == "기쁨"), None
            )
            other_emotion = next(
                (e for e in level2_emotions if e["emotion"] == "기타감정"), None
            )

            if joy_emotion and other_emotion:
                combination_type = "ambivalent"  # 양가감정
                details = {
                    "positive_aspect": {
                        "emotion": "기쁨",
                        "strength": joy_emotion["probability"],
                    },
                    "negative_aspect": {
                        "emotions": level3_emotions,
                        "primary": (
                            max(level3_emotions, key=lambda x: x["probability"])
                            if level3_emotions
                            else None
                        ),
                    },
                    "conflict_intensity": abs(
                        joy_emotion["probability"] - other_emotion["probability"]
                    ),
                }

        return {"type": combination_type, "details": details}

## 챗봇 생성

In [11]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_core.prompts import MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
import yaml
import os


class EmotionChatbot:
    def __init__(
        self,
        model_dir,
        confidence_threshold=0.6,
        emotion_threshold=0.3,
        centers_csv_path=None,  # 추가 : CSV 파일 경로
        enable_safety_features=True,  # 안전 기능 활성화 옵션
    ):
        # 감정 분류기 초기화
        self.classifier = HierarchicalEmotionClassifier(
            model_dir,
            confidence_threshold=confidence_threshold,
            emotion_threshold=emotion_threshold,
        )

        # 임계값을 챗봇에서도 저장 (필요시 사용)
        self.confidence_threshold = confidence_threshold
        self.emotion_threshold = emotion_threshold

        # 프롬프트 로드
        self.prompts = self._load_prompts()

        # 다중 llm 초기화
        self.llms = self._initialize_all_llms()
        self.shared_memory = None  # 공유 메모리 변수 추가

        # 모델별 메모리와 체인 설정
        self._setup_model_chains()

        # llm 선정을 위해 여러 API 키를 환경 변수에서 불러옴

        """
        try:
            # API 키 가져오기
            api_key = os.getenv("OPENAI_API_KEY")
            api_key2 = os.getenv("ANTHROPIC_API_KEY")

            if llm_provider == "openai":
                if not api_key:
                    raise ValueError("OPENAI_API_KEY 환경변수가 설정되지 않았습니다.")

                self.llm = ChatOpenAI(
                    model="gpt-4o", temperature=0.4, max_tokens=1000, api_key=api_key
                )

            elif llm_provider == "claude":
                if not api_key2:
                    raise ValueError(
                        "ANTHROPIC_API_KEY 환경변수가 설정되지 않았습니다."
                    )

                self.llm = ChatAnthropic(
                    model="claude-opus-4-20250514",
                    temperature=0.4,
                    max_tokens=1000,
                    api_key=api_key2,
                )

            elif llm_provider == "gemini":
                try:
                    if not api_key3:
                        raise ValueError("GEMINI_API_KEY 환경변수가 설정되지 않았습니다.")
                    
                    print(f"Gemini API key 존재: {bool(api_key3)}")
                    print("ChatGoogleGenerativeAI 초기화 시도...")
                    
                    self.llm = ChatGoogleGenerativeAI(
                        model="gemini-1.5-pro-latest",
                        temperature=0.4,
                        api_key=api_key3,
                        

                    )
                    print("ChatGoogleGenerativeAI 초기화 완료")
        
                except Exception as gemini_error:
                    print(f"Gemini 초기화 중 오류: {gemini_error}")
                    raise

            # 메모리 초기화
            self.shared_memory = ConversationSummaryBufferMemory(
                llm=self.llm,
                return_messages=True,
                max_token_limit=2000,
                memory_key="chat_history",
            )

            # 기본 프롬프트 설정
            self.system_prompt_text = self.prompts.get("basic_system")
            if not self.system_prompt_text:
                raise ValueError(
                    "basic_system 프롬프트가 로드되지 않았습니다. prompts/basic_system.yaml 파일을 확인해주세요."
                )

            self.chat_prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", self.system_prompt_text),
                    MessagesPlaceholder(variable_name="chat_history"),
                    ("human", "{message}"),
                ]
            )

            self.chain = self.chat_prompt | self.llm
            print("✅ 챗봇 초기화 완료")

        except Exception as e:
            print(f"❌ 챗봇 초기화 중 오류 발생: {e}")
            raise
        """

        # 안전 기능 추가 부분
        if enable_safety_features:
            # 위험 키워드 감지기 추가
            self.risk_detector = RiskKeywordDetector()

            # CSV 파일에서 센터 정보 로드
            if centers_csv_path and os.path.exists(centers_csv_path):
                self.center_matcher = LocationBasedCenterMatcher(centers_csv_path)
            else:
                # 기본 센터 사용
                self.center_matcher = LocationBasedCenterMatcher()
            # 위치 처리 핸들러
            self.location_handler = LocationHandler(self.center_matcher)

            # 청소년 언어 처리기
            self.youth_processor = YouthLanguageProcessor()

            # 위험 상황 로그
            self.risk_log = []

            print("✅ 청소년 안전 기능 활성화됨")

        else:
            # 안전 기능 비활성화
            self.risk_detector = None
            self.center_matcher = None
            self.location_handler = None
            self.youth_processor = None
            self.risk_log = []

            print("⚠️ 청소년 안전 기능 비활성화됨")

    def _load_prompts(self):
        """YAML 파일에서 프롬프트들을 로드"""
        prompts = {}

        # 프롬프트 파일들 리스트 (복합 감정 프롬프트만)
        prompt_files = {
            "basic_system": "prompts/basic_system.yaml",
            "ambivalent_empathy": "prompts/ambivalent_empathy.yaml",
            "mixed_validation": "prompts/mixed_validation.yaml",
            "exploratory_empathy": "prompts/exploratory_empathy.yaml",
            "direct_empathy": "prompts/direct_empathy.yaml",
            "general_conversation": "prompts/general_conversation.yaml",
        }

        for prompt_name, file_path in prompt_files.items():
            try:
                if os.path.exists(file_path):
                    with open(file_path, "r", encoding="utf-8") as f:
                        data = yaml.safe_load(f)
                        # YAML 파일에서 'template' 키를 찾아서 사용
                        if isinstance(data, dict) and "template" in data:
                            prompts[prompt_name] = data["template"]
                        elif isinstance(data, str):
                            prompts[prompt_name] = data
                        else:
                            print(
                                f"⚠️ {file_path}에서 올바른 템플릿을 찾을 수 없습니다."
                            )
                else:
                    print(f"⚠️ {file_path} 파일이 존재하지 않습니다.")
            except Exception as e:
                print(f"❌ {file_path} 로드 중 오류: {e}")

        return prompts

    def _initialize_all_llms(self):
        # 모든 LLM 한번에 초기화
        llms = {}

        try:
            if openai_key:
                # 일반 대화용
                llms["gpt35"] = ChatOpenAI(
                    model="gpt-3.5-turbo",
                    temperature=0.6,
                    max_tokens=800,
                    api_key=openai_key,
                )
            print("🌟일반대화용 GPT3.5 초기화완료")

            if openai_key:
                # 감정 대화용
                llms["gpt4o"] = ChatOpenAI(
                    model="gpt-4o", temperature=0.3, max_tokens=1200, api_key=openai_key
                )
            print("🌟감정대화용 GPT4 초기화완료")

            if anthropic_key:
                # 위험 키워드 탐지
                llms["claude"] = ChatAnthropic(
                    model="claude-opus-4-20250514",
                    temperature=0.3,
                    max_tokens=1200,
                    api_key=anthropic_key,
                )
            print("🌟위험대화용 Cluade 초기화완료")

        except Exception as e:
            print(f"🥺 LLM 초기화 오류{e}")
            raise
        return llms

    def _choose_model(self, text):
        # 텍스트 분석후 적절한 모델 선택

        # 1. 감정 분석
        emotion_result = self.classify_emotion(text)

        # 2. 위험도 분석 (안전 기능이 있을 때만)
        risk_analysis = None
        if hasattr(self, "risk_detector") and self.risk_detector:
            risk_analysis = self.risk_detector.detect_risk(text)

        # 3. 모델 선택 로직
        # 위험상황이면 Claude 우선
        if risk_analysis and risk_analysis["requires_intervention"]:
            if "claude" in self.llms:
                return "claude", emotion_result, risk_analysis
            elif "gpt4o" in self.llms:  # Claude 없으면 GPT-4o로 폴백
                return "gpt4o", emotion_result, risk_analysis

        # 일반대화면 GPT-3.5
        emotion = emotion_result["final"]["prediction"]
        confidence = emotion_result["final"]["confidence"]

        if emotion == "일반대화" and confidence > 0.7:
            if "gpt35" in self.llms:
                return "gpt35", emotion_result, risk_analysis

        # 감정관련이면 GPT-4o
        if "gpt4o" in self.llms:
            return "gpt4o", emotion_result, risk_analysis

        # 폴백: 사용 가능한 첫 번째 모델
        available_model = list(self.llms.keys())[0]
        return available_model, emotion_result, risk_analysis

    def generate_smart_response(self, text):
        """상황에 맞는 모델로 스마트 응답 생성"""

        try:
            # 1. 모델 선택
            selected_model, emotion_result, risk_analysis = self._choose_model(text)

            print(f"🤖 선택된 모델: {selected_model}")

            # 2. 해당 모델로 응답 생성
            response = self._generate_with_selected_model(
                text, selected_model, emotion_result, risk_analysis
            )

            return {
                "ai_response": response,
                "model_used": selected_model,
                "emotion_analysis": emotion_result,
                "risk_analysis": risk_analysis,
            }

        except Exception as e:
            print(f"❌ 응답 생성 오류: {e}")
            # 오류시 기본 모델로 폴백
            fallback_model = list(self.llms.keys())[0]
            fallback_response = (
                f"죄송합니다. 응답 생성 중 문제가 발생했습니다: {str(e)}"
            )

            return {
                "ai_response": fallback_response,
                "model_used": fallback_model + " (fallback)",
                "error": str(e),
            }

    def _generate_with_selected_model(
        self, text, model_name, emotion_result, risk_analysis
    ):
        """선택된 모델로 실제 응답 생성"""

        # 공유 메모리에서 대화 기록 가져오기
        chat_history = self.shared_memory.chat_memory.messages
        llm = self.llms[model_name]

        # 🔧 emotion_info 준비 (모든 경우에 사용)
        emotion_info = f"감정: {emotion_result['final']['prediction']}, 신뢰도: {emotion_result['final']['confidence']:.2f}"

        # 위험상황이고 Claude면 특별 처리
        if (
            risk_analysis
            and risk_analysis["requires_intervention"]
            and model_name == "claude"
        ):
            response = self._generate_crisis_response_safe(
                text, risk_analysis, [], emotion_result
            )

        elif emotion_result["final"]["prediction"] == "일반대화":
            # 🔧 일반대화: emotion_info 전달 추가
            prompt = ChatPromptTemplate.from_messages(
                [
                    (
                        "system",
                        self.prompts.get(
                            "general_conversation", "당신은 친근한 AI 친구입니다."
                        ),
                    ),
                    MessagesPlaceholder(variable_name="chat_history"),
                    ("human", "{message}"),
                ]
            )

            chain = prompt | llm
            llm_response = chain.invoke(
                {
                    "message": text,
                    "emotion_info": emotion_info,  # 🔧 추가
                    "chat_history": chat_history,
                }
            )

            response = (
                llm_response.content
                if hasattr(llm_response, "content")
                else str(llm_response)
            )

        else:
            # 감정관련: 기존 로직 사용하되 emotion_info 전달
            response = self._generate_emotion_response_with_model(
                text, emotion_result, chat_history, llm
            )

        # 공유 메모리에 대화 저장
        self.shared_memory.chat_memory.add_user_message(text)
        self.shared_memory.chat_memory.add_ai_message(response)

        return response

    def _generate_emotion_response_with_model(
        self, text, emotion_result, chat_history, llm
    ):
        """감정 응답 생성 (기존 로직 활용)"""

        # 기본 감정 프롬프트 사용
        system_prompt = self.prompts.get(
            "basic_system", "당신은 공감적인 상담사입니다."
        )

        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                MessagesPlaceholder(variable_name="chat_history"),
                ("human", "{message}"),
            ]
        )

        chain = prompt | llm

        # 🔧 감정 정보 준비
        emotion_info = f"감정: {emotion_result['final']['prediction']}, 신뢰도: {emotion_result['final']['confidence']:.2f}"

        llm_response = chain.invoke(
            {
                "message": text,
                "emotion_info": emotion_info,  # 🔧 추가
                "chat_history": chat_history,
            }
        )

        return (
            llm_response.content
            if hasattr(llm_response, "content")
            else str(llm_response)
        )

    def _setup_model_chains(self):
        """공유메모리 설정"""

        # 대표 모델 선택
        if "gpt4o" in self.llms:
            primary_llm = self.llms["gpt4o"]
        elif "claude" in self.llms:
            primary_llm = self.llms["claude"]
        else:
            primary_llm = list(self.llms.values())[0]

        self.llm = primary_llm

        # 공유메모리 생성
        self.shared_memory = ConversationSummaryBufferMemory(
            llm=primary_llm,
            return_messages=True,
            max_token_limit=2000,
            memory_key="chat_history",
        )

        print("공유 메모리 설정 완료")

        self.system_prompt_text = self.prompts.get("basic_system")

        self.chat_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", self.system_prompt_text),
                MessagesPlaceholder(variable_name="chat_history"),
                ("human", "{message}"),
            ]
        )

        self.chain = self.chat_prompt | self.llm

        print("공유 메모리 및 기본 체인 설정 완료")

    def classify_emotion(self, text):
        """기본 감정 분류"""
        return self.classifier.predict_hierarchical(text)

    def format_analysis(self, result):
        """기본 분석 결과 포맷팅"""
        output = f"""
📝 입력: {result['original_text']}
🔍 전처리: {result['preprocessed_text']}

🎯 최종 결과: {result['final']['prediction']} (신뢰도: {result['final']['confidence']:.4f})
📊 예측 경로: {' → '.join(result['path'])}

📈 단계별 분석:
"""

        for level, data in result["levels"].items():
            output += f"\n🔸 {data['step']}\n"
            output += f"   결과: {data['prediction']} ({data['confidence']:.4f})\n"
            output += f"   확률: "
            for emotion, prob in data["probabilities"].items():
                output += f"{emotion}({prob:.3f}) "
            output += "\n"

        return output

    def generate_response(self, text):
        """기본 챗봇 응답 생성 (메모리 추가)"""
        try:
            # 기본 감정 분류
            emotion_result = self.classify_emotion(text)

            # 감정 분석 포맷팅
            emotion_info = f"감정: {emotion_result['final']['prediction']}, 신뢰도: {emotion_result['final']['confidence']:.2f}, 경로: {' → '.join(emotion_result['path'])}"

            # 메모리에서 대화 기록 가져오기
            chat_history = self.shared_memory.chat_memory.messages

            # 체인 실행
            response = self.chain.invoke(
                {
                    "message": text,
                    "emotion_info": emotion_info,
                    "chat_history": chat_history,
                }
            )

            # 응답 추출
            if hasattr(response, "content"):
                ai_response = response.content

                # 메모리에 대화저장
                self.shared_memory.chat_memory.add_user_message(text)
                self.shared_memory.chat_memory.add_ai_message(ai_response)

                return ai_response
            else:
                return "응답생성에 실패했습니다"

        except Exception as e:
            print(f"응답 생성 중 오류 발생: {e}")
            return "응답 생성 중 오류가 발생했습니다. 다시 시도해주세요."

    def generate_response_with_emotion_detection(self, text):
        """ "감정 분류에 따라 응답 생성"""
        try:
            emotion_result = self.classify_emotion(text)

            chat_history = self.shared_memory.chat_memory.messages

            # 최종 예측 결과 확인
            finally_prediction = emotion_result["final"]["prediction"]

            if finally_prediction == "일반대화":
                # 일반 대화 프롬프트 사용
                response = self.generate_general_conversation_response(
                    text, emotion_result, chat_history
                )
            else:
                # 감정이라면
                response = self.generate_emotion_response(
                    text, emotion_result, chat_history
                )
            return response
        except Exception as e:
            print(f"감정 분류 및 응답 생성 중 오류 발생: {e}")
            return "죄송합니다. 응답 생성 중 문제가 발생했네요. 다시 말씀해 주시겠어요?"

    # 일반대화 생성
    def generate_general_conversation_response(
        self, text, emotion_result, chat_history
    ):
        """일반 대화 응답 생성"""
        try:
            # 🔧 감정 정보 준비
            emotion_info = f"감정: {emotion_result['final']['prediction']}, 신뢰도: {emotion_result['final']['confidence']:.2f}"

            # 🔧 일반대화 프롬프트 구성
            if "gpt35" in self.llms:
                llm = self.llms["gpt35"]
            else:
                llm = list(self.llms.values())[0]  # 폴백

            general_chat_prompt = ChatPromptTemplate.from_messages(
                [
                    (
                        "system",
                        self.prompts.get(
                            "general_conversation", "당신은 친근한 AI 친구입니다."
                        ),
                    ),
                    MessagesPlaceholder(variable_name="chat_history"),
                    ("human", "{message}"),
                ]
            )

            # 체인 실행
            chain = general_chat_prompt | llm
            response = chain.invoke(
                {
                    "message": text,
                    "emotion_info": emotion_info,  # 🔧 추가
                    "chat_history": chat_history,
                }
            )

            if hasattr(response, "content"):
                ai_response = response.content

                # 🔧 shared_memory에 대화 저장
                self.shared_memory.chat_memory.add_user_message(text)
                self.shared_memory.chat_memory.add_ai_message(ai_response)

                return ai_response
            else:
                return "응답 생성에 실패했습니다."

        except Exception as e:
            print(f"일반 대화 응답 생성 중 오류 발생: {e}")
            return "죄송합니다. 일반 대화 응답 생성 중 문제가 발생했네요. 다시 말씀해 주시겠어요?"

    def generate_empathetic_response(self, text):
        """복합 감정을 고려한 공감 응답 생성"""
        try:
            # 복합 감정 분석
            emotion_analysis = self.classifier.predict_with_confidence_analysis(text)
            multi_emotion_result = self.classifier.predict_multi_emotion_threshold(text)

            # 응답 전략 결정
            response_strategy = self._determine_response_strategy(
                emotion_analysis, multi_emotion_result
            )

            # 프롬프트에 복합 감정 정보 추가
            emotion_context = self._format_emotion_context(
                emotion_analysis, multi_emotion_result, response_strategy
            )

            # 메모리에서 대화 기록 가져오기
            chat_history = self.shared_memory.chat_memory.messages

            # 개선된 프롬프트로 응답 생성
            response = self._generate_response_with_complex_emotion_context(
                text, emotion_context, chat_history
            )

            return response

        except Exception as e:
            print(f"공감 응답 생성 중 오류: {e}")
            return "죄송합니다. 응답 생성 중 문제가 발생했네요. 다시 말씀해 주시겠어요?"

    def _determine_response_strategy(self, emotion_analysis, multi_emotion_result):
        """응답 전략 결정"""
        if emotion_analysis["complexity_level"] == "mixed":
            if multi_emotion_result["emotion_combination"]["type"] == "ambivalent":
                return "ambivalent_empathy"  # 양가감정 공감
            else:
                return "mixed_validation"  # 혼합감정 인정
        elif emotion_analysis["complexity_level"] == "complex":
            return "exploratory_empathy"  # 탐색적 공감
        else:
            return "direct_empathy"  # 직접적 공감

    def _format_emotion_context(self, emotion_analysis, multi_emotion_result, strategy):
        """감정 맥락 포맷팅"""
        context = {
            "primary_emotion": emotion_analysis["primary_emotion"],
            "confidence": emotion_analysis["primary_confidence"],
            "complexity": emotion_analysis["complexity_level"],
            "response_strategy": strategy,
        }

        # 복합 감정 정보 추가
        if emotion_analysis["mixed_emotions"]:
            context["mixed_emotions"] = emotion_analysis["mixed_emotions"]

        # 양가감정 정보 추가
        if multi_emotion_result["emotion_combination"]["type"] == "ambivalent":
            context["ambivalent_details"] = multi_emotion_result["emotion_combination"][
                "details"
            ]

        return context

    def _generate_response_with_complex_emotion_context(
        self, text, emotion_context, chat_history
    ):
        """복합 감정 맥락을 고려한 응답 생성"""

        # 필수 프롬프트들이 모두 로드되었는지 확인
        required_prompts = [
            "ambivalent_empathy",
            "mixed_validation",
            "exploratory_empathy",
            "direct_empathy",
        ]
        missing_prompts = [
            p for p in required_prompts if p not in self.prompts or not self.prompts[p]
        ]

        if missing_prompts:
            raise ValueError(f"필수 프롬프트가 누락되었습니다: {missing_prompts}")

        # YAML에서 로드된 프롬프트 사용
        strategy_prompts = {}
        for strategy in required_prompts:
            strategy_prompts[strategy] = self.prompts[strategy]

        # 감정 정보 문자열 생성
        emotion_info_str = f"""
감정 분석 결과:
- 주요 감정: {emotion_context['primary_emotion']} (신뢰도: {emotion_context['confidence']:.2f})
- 복잡도: {emotion_context['complexity']}
- 응답 전략: {emotion_context['response_strategy']}
"""

        if "mixed_emotions" in emotion_context:
            emotion_info_str += "\n혼합 감정들:\n"
            for emotion in emotion_context["mixed_emotions"]:
                emotion_info_str += (
                    f"- {emotion['emotion']}: {emotion['confidence']:.2f}\n"
                )

        if "ambivalent_details" in emotion_context:
            details = emotion_context["ambivalent_details"]
            emotion_info_str += f"\n양가감정 세부사항:\n"
            emotion_info_str += f"- 긍정적 측면: {details['positive_aspect']['emotion']} ({details['positive_aspect']['strength']:.2f})\n"
            if details["negative_aspect"]["primary"]:
                emotion_info_str += f"- 부정적 측면: {details['negative_aspect']['primary']['emotion']} ({details['negative_aspect']['primary']['probability']:.2f})\n"

        # 전략에 맞는 시스템 프롬프트 선택
        system_prompt = strategy_prompts.get(
            emotion_context["response_strategy"], strategy_prompts["direct_empathy"]
        )

        # 프롬프트 구성
        chat_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                MessagesPlaceholder(variable_name="chat_history"),
                ("human", "{message}"),
            ]
        )

        # 체인 실행
        chain = chat_prompt | self.llm
        response = chain.invoke(
            {
                "message": text,
                "chat_history": chat_history,
                "emotion_info": emotion_info_str,
            }
        )

        if hasattr(response, "content"):
            ai_response = response.content
            # 메모리에 대화 저장
            self.shared_memory.chat_memory.add_user_message(text)
            self.shared_memory.chat_memory.add_ai_message(ai_response)
            return ai_response
        else:
            return "응답 생성에 실패했습니다."

    def reload_prompts(self):
        """프롬프트 재로드"""
        try:
            self.prompts = self._load_prompts()
            self.system_prompt_text = self.prompts.get("basic_system")
            if not self.system_prompt_text:
                print("⚠️ basic_system 프롬프트가 로드되지 않았습니다.")
                self.system_prompt_text = "당신은 공감적인 AI 상담사입니다."

            # 🔧 기본 체인 재생성 (self.llm 사용)
            self.chat_prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", self.system_prompt_text),
                    MessagesPlaceholder(variable_name="chat_history"),
                    ("human", "{message}"),
                ]
            )
            self.chain = self.chat_prompt | self.llm  # self.llm 사용

            print("✅ 프롬프트가 성공적으로 재로드되었습니다.")
        except Exception as e:
            print(f"❌ 프롬프트 재로드 중 오류: {e}")

    def process_message(self, text, include_ai_response=True):
        """기본 메시지 전체 처리"""
        # 감정 분석
        emotion_result = self.classify_emotion(text)
        emotion_analysis = self.format_analysis(emotion_result)

        result = {
            "emotion_analysis": emotion_analysis,
            "emotion_result": emotion_result,
        }

        # AI 응답
        if include_ai_response:
            ai_response = self.generate_response(text)
            result["ai_response"] = ai_response
            result["full_response"] = f"{emotion_analysis}\n💬 AI 상담:\n{ai_response}"
        else:
            result["full_response"] = emotion_analysis

        return result

    def process_complex_emotion_message(self, text):
        """복합 감정을 고려한 메시지 처리"""
        # 기본 감정 분석
        basic_result = self.classify_emotion(text)

        # 복합 감정 분석
        confidence_analysis = self.classifier.predict_with_confidence_analysis(text)
        multi_emotion_analysis = self.classifier.predict_multi_emotion_threshold(text)

        # 공감 응답 생성
        ai_response = self.generate_empathetic_response(text)

        # 결과 통합
        result = {
            "basic_emotion_analysis": self.format_analysis(basic_result),
            "confidence_analysis": confidence_analysis,
            "multi_emotion_analysis": multi_emotion_analysis,
            "ai_response": ai_response,
            "complexity_insight": self._generate_complexity_insight(
                confidence_analysis, multi_emotion_analysis
            ),
            "full_response": f"""
📊 감정 복잡성 분석:
{chr(10).join(self._generate_complexity_insight(confidence_analysis, multi_emotion_analysis))}

💬 AI 상담:
{ai_response}
""",
        }

        return result

    def _generate_complexity_insight(self, confidence_analysis, multi_emotion_analysis):
        """복잡성 인사이트 생성"""
        insights = []

        if confidence_analysis["complexity_level"] == "mixed":
            insights.append("🔀 혼합된 감정이 감지되었습니다.")

        if multi_emotion_analysis["emotion_combination"]["type"] == "ambivalent":
            insights.append("⚖️ 상반된 감정이 동시에 나타나는 양가감정 상태입니다.")

        if confidence_analysis["primary_confidence"] < 0.6:
            insights.append("🌊 감정이 복잡하고 미묘한 상태로 보입니다.")

        if len(multi_emotion_analysis["detected_emotions"]) > 3:
            insights.append("🎭 다양한 감정이 복합적으로 나타나고 있습니다.")

        return insights if insights else ["💡 비교적 명확한 감정 상태입니다."]

    # 메모리 관련 유틸리티 메서드들
    def get_conversation_summary(self):
        """현재 대화 요약 가져오기"""
        try:
            return self.shared_memory.predict_new_summary(
                self.shared_memory.chat_memory.messages, ""
            )
        except Exception as e:
            print(f"대화 요약 생성 중 오류: {e}")
            return "요약을 생성할 수 없습니다."

    def get_chat_history(self):
        """전체 대화 기록 가져오기"""
        if hasattr(self, "shared_memory") and self.shared_memory:
            return self.shared_memory.chat_memory.messages
        else:
            return []

    def clear_memory(self):
        """메모리 초기화"""
        if hasattr(self, "shared_memory") and self.shared_memory:
            self.shared_memory.clear()
            print("💭 공유 대화 기록이 초기화되었습니다.")
        else:
            print("⚠️ 초기화할 메모리가 없습니다.")

    def get_memory_status(self):
        """메모리 상태 확인"""
        try:
            messages = self.shared_memory.chat_memory.messages
            token_count = self.shared_memory.llm.get_num_tokens_from_messages(messages)

            return {
                "message_count": len(messages),
                "token_count": token_count,
                "max_token_limit": self.shared_memory.max_token_limit,
                "is_summarizing": token_count > self.shared_memory.max_token_limit,
            }
        except Exception as e:
            print(f"메모리 상태 확인 중 오류: {e}")
            return {
                "message_count": 0,
                "token_count": 0,
                "max_token_limit": 2000,
                "is_summarizing": False,
                "error": str(e),
            }

    def set_thresholds(self, confidence_threshold=None, emotion_threshold=None):
        """임계값 동적 변경"""
        if confidence_threshold is not None:
            self.confidence_threshold = confidence_threshold
            self.classifier.confidence_threshold = confidence_threshold
            print(f"신뢰도 임계값이 {confidence_threshold}로 변경되었습니다.")

        if emotion_threshold is not None:
            self.emotion_threshold = emotion_threshold
            self.classifier.emotion_threshold = emotion_threshold
            print(f"감정 탐지 임계값이 {emotion_threshold}로 변경되었습니다.")

    def get_thresholds(self):
        """현재 임계값 조회"""
        return {
            "confidence_threshold": self.confidence_threshold,
            "emotion_threshold": self.emotion_threshold,
        }

    def generate_emotion_response(self, text, emotion_result, chat_history):
        """감정 관련 프롬프트로 응답 생성 (기존 로직)"""
        try:
            # 감정 분석 포맷팅
            emotion_info = f"감정: {emotion_result['final']['prediction']}, 신뢰도: {emotion_result['final']['confidence']:.2f}, 경로: {' → '.join(emotion_result['path'])}"

            # 기본 감정 프롬프트 사용
            response = self.chain.invoke(
                {
                    "message": text,
                    "emotion_info": emotion_info,
                    "chat_history": chat_history,
                }
            )

            if hasattr(response, "content"):
                ai_response = response.content

                # 메모리에 대화 저장
                self.shared_memory.chat_memory.add_user_message(text)
                self.shared_memory.chat_memory.add_ai_message(ai_response)

                return ai_response
            else:
                return "응답 생성에 실패했습니다"

        except Exception as e:
            print(f"감정 응답 생성 중 오류: {e}")
            return "죄송합니다. 감정 분석 응답 생성 중 문제가 발생했습니다."

    def process_message_with_detection(self, text, include_ai_response=True):
        """일반대화/감정 구분하여 메시지 처리"""
        # 감정 분석
        emotion_result = self.classify_emotion(text)
        emotion_analysis = self.format_analysis(emotion_result)

        result = {
            "emotion_analysis": emotion_analysis,
            "emotion_result": emotion_result,
            "conversation_type": emotion_result["final"][
                "prediction"
            ],  # 일반대화 vs 감정
        }

        # AI 응답
        if include_ai_response:
            ai_response = self.generate_response_with_emotion_detection(text)
            result["ai_response"] = ai_response
            result["full_response"] = (
                f"{emotion_analysis}\n\n💬 AI 응답:\n{ai_response}"
            )
        else:
            result["full_response"] = emotion_analysis

        return result

        # 메서드 추가

    def process_youth_message(
        self,
        text: str,
        user_id: str = "default",
        latitude: float = None,
        longitude: float = None,
    ) -> Dict:
        # 안전 기능이 비활성화되어있으면 기본 처리
        if not hasattr(self, "risk_detector") or self.risk_detector is None:
            return self.process_message(text, include_ai_response=True)

        try:
            # 1. 기본 감정 분석:
            emotion_result = self.classify_emotion(text)

            # 2. 위험 키워드 감지
            risk_analysis = self.risk_detector.detect_risk(text)

            # 3. 위험 정보 처리
            location_result = self.location_handler.process_user_location(
                user_id=user_id,
                latitude=latitude,
                longitude=longitude,
                location_text=text,
            )

            # 4. 청소년 언어 분석
            youth_analysis = self.youth_processor.process_youth_message(text)

            # 5. 감정-위험도 일관성 체크 추가
            consistency_check = self._check_emotion_risk_consistency(
                emotion_result, risk_analysis
            )

            # 6. 최종 위험도 조정
            final_risk_analysis = self._adjust_final_risk_level(
                risk_analysis, emotion_result, consistency_check
            )

            # 7. 근처 센터 찾기 (조정된 위험도 기준)
            nearby_centers = []
            if (
                final_risk_analysis["requires_intervention"]
                and location_result["coordinates"]
            ):
                nearby_centers = self.center_matcher.find_nearest_centers(
                    location_result["coordinates"], max_distance=50.0, limit=3
                )

            # 8. 응답 생성 (조정된 위험도에 따라 다른 전략)
            if final_risk_analysis["requires_intervention"]:
                ai_response = self._generate_crisis_response_safe(
                    text, final_risk_analysis, nearby_centers, emotion_result
                )
                # 위험 상황 로깅
                self._log_risk_situation_safe(
                    text, final_risk_analysis, location_result, user_id
                )
            else:
                ai_response = self._generate_safe_response(
                    text, emotion_result, youth_analysis
                )

            # 9. 결과 통합
            result = {
                "original_text": text,
                "user_id": user_id,
                "emotion_analysis": emotion_result,
                "risk_analysis": risk_analysis,
                "final_risk_analysis": final_risk_analysis,
                "consistency_check": consistency_check,
                "youth_analysis": youth_analysis,
                "location_info": location_result,
                "nearby_centers": nearby_centers,
                "ai_response": ai_response,
                "requires_immediate_help": final_risk_analysis["requires_intervention"],
                "timestamp": datetime.now(),
            }

            return result

        except Exception as e:  # 🆕 이 부분 추가!
            print(f"❌ 청소년 메시지 처리 중 오류: {e}")
            # 오류 시 기본 처리로 폴백
            return self.process_message(text, include_ai_response=True)

    def _generate_crisis_response_safe(
        self,
        text: str,
        risk_analysis: Dict,
        nearby_centers: List[Dict],
        emotion_result: Dict,
    ) -> str:
        """🆕 위기 상황 안전 응답 생성"""

        # 기본 공감 메시지
        crisis_message = f"💙 {risk_analysis.get('risk_message', '많이 힘드시군요. 혼자가 아니에요.')}\n\n"

        # 근처 센터 정보 추가
        if nearby_centers:
            crisis_message += "🏥 **가까운 도움받을 곳:**\n"
            for i, center in enumerate(nearby_centers[:2], 1):
                crisis_message += f"{i}. {center['name']}\n"
                crisis_message += f"   📞 {center['phone']}\n"
                if "distance" in center:
                    crisis_message += f"   🚗 거리: {center['distance']}km\n"
                crisis_message += "\n"

        # 응급 연락처
        crisis_message += """🚨 **즉시 도움이 필요하다면:**
• 응급상황: 112
• 청소년전화: 1388 (24시간)
• 자살예방상담: 1393
• 위기상담: 1577-0199

혼자가 아니라는 걸 기억해주세요. 💙"""

        return crisis_message

    def _generate_safe_response(
        self, text: str, emotion_result: Dict, youth_analysis: Dict
    ) -> str:
        """개선된 일반 상황 안전 응답 생성"""

        # 감정 결과 확인
        emotion = emotion_result["final"]["prediction"]
        confidence = emotion_result["final"]["confidence"]

        # 긍정적 감정일 때
        if emotion in ["기쁨", "일반대화"] and confidence > 0.6:
            if youth_analysis.get("language_style") == "youth_casual":
                response_start = "오늘 기분 좋으시다니 완전 좋네요! 😊 "
            else:
                response_start = "기분이 좋으시다니 정말 다행이에요! 😊 "
        else:
            # 기존 로직
            if youth_analysis.get("language_style") == "youth_casual":
                response_start = "그런 마음 완전 이해해💙 "
            else:
                response_start = "힘든 마음을 말씀해주셨군요. "

        # 기존 generate_response 호출하되 앞에 친화적 메시지 추가
        base_response = self.generate_response(text)

        return response_start + base_response

    def _log_risk_situation_safe(
        self, text: str, risk_analysis: Dict, location_result: Dict, user_id: str
    ):
        """🆕 위험 상황 안전 로깅"""
        log_entry = {
            "timestamp": datetime.now(),
            "user_id": user_id,
            "risk_level": risk_analysis["risk_level"],
            "detected_keywords": [
                r["keyword"] for r in risk_analysis.get("detected_risks", [])
            ],
            "has_location": location_result.get("coordinates") is not None,
            "intervention_provided": True,
        }

        self.risk_log.append(log_entry)

        # 로그가 너무 길어지면 정리 (최근 50개만 유지)
        if len(self.risk_log) > 100:
            self.risk_log = self.risk_log[-50:]

    def _check_emotion_risk_consistency(
        self, emotion_result: Dict, risk_analysis: Dict
    ) -> Dict:
        """감정과 위험도의 일관성 체크"""
        emotion = emotion_result["final"]["prediction"]
        risk_level = risk_analysis["risk_level"]
        has_positive_context = risk_analysis.get("has_positive_context", False)

        # 일관성 점수 계산
        consistency_score = 1.0
        warnings = []

        # 긍정적 감정 + 높은 위험도 = 비일관적
        if emotion in ["기쁨", "일반대화"] and risk_level in ["high", "critical"]:
            if not has_positive_context:
                consistency_score -= 0.5
                warnings.append("긍정적 감정과 높은 위험도 불일치")

        return {
            "consistency_score": max(0.0, min(1.0, consistency_score)),
            "warnings": warnings,
        }

    def _adjust_final_risk_level(
        self, risk_analysis: Dict, emotion_result: Dict, consistency: Dict
    ) -> Dict:
        """최종 위험도 조정"""
        base_risk = risk_analysis["risk_level"]
        emotion = emotion_result["final"]["prediction"]
        confidence = emotion_result["final"]["confidence"]

        # 일관성이 낮으면 위험도 재평가
        if consistency["consistency_score"] < 0.5:
            # 긍정적 감정이 확실하면 위험도 낮춤
            if emotion in ["기쁨", "일반대화"] and confidence > 0.7:
                if base_risk == "high":
                    base_risk = "medium"
                elif base_risk == "medium":
                    base_risk = "low"

        return {
            "risk_level": base_risk,
            "requires_intervention": base_risk in ["critical", "high"],
            "adjustment_reason": f"감정 일관성 고려",
            "original_risk": risk_analysis["risk_level"],
        }

# 테스트

In [12]:
from dataclasses import dataclass
from datetime import datetime
import os
import pandas as pd
from geopy.distance import geodesic


@dataclass
class SupportCenter:
    """지원센터 정보"""

    region: str
    name: str
    address: str
    phone: str
    website: str
    latitude: float
    longitude: float
    center_type: str


class RiskKeywordDetector:
    """개선된 위험 키워드 감지기"""

    def __init__(self):
        # 위험 키워드를 더 정확하게 정의
        self.risk_patterns = {
            "critical": {
                "patterns": [
                    r"죽고\s*싶",
                    r"자살",
                    r"자해",
                    r"사라지고\s*싶",
                    r"죽어야지",
                    r"죽을래",
                    r"죽었으면",
                ],
                "exclusions": [],
            },
            "high": {
                "patterns": [
                    r"가출하고\s*싶",
                    r"집에\s*안\s*가고\s*싶",
                    r"때리고\s*싶",
                    r"부모\s*때리",
                    r"선생님\s*때리",
                ],
                "exclusions": [r"기분\s*좋", r"행복", r"즐거"],
            },
            "medium": {
                "patterns": [
                    r"너무\s*우울",
                    r"심한\s*불안",
                    r"많이\s*힘들",
                    r"극심한\s*스트레스",
                    r"견딜\s*수\s*없",
                ],
                "exclusions": [r"기분\s*좋", r"괜찮", r"좋아", r"행복"],
            },
        }

        # 긍정적 맥락 키워드
        self.positive_context = [
            r"기분\s*좋",
            r"행복",
            r"즐거",
            r"괜찮",
            r"좋아",
            r"재밌",
            r"신나",
            r"웃",
            r"밝",
            r"희망",
        ]

    def detect_risk(self, text: str) -> dict:
        """개선된 위험도 감지"""
        text_processed = text.lower().strip()
        detected_risks = []
        max_risk_level = "low"

        # 1. 긍정적 맥락 체크
        has_positive_context = self._check_positive_context(text_processed)

        # 2. 각 위험 레벨별로 패턴 매칭
        for level, patterns_data in self.risk_patterns.items():
            patterns = patterns_data["patterns"]
            exclusions = patterns_data["exclusions"]

            for pattern in patterns:
                matches = re.findall(pattern, text_processed)
                if matches:
                    # 제외 패턴 체크
                    is_excluded = False
                    for exclusion in exclusions:
                        if re.search(exclusion, text_processed):
                            is_excluded = True
                            break

                    # 긍정적 맥락이 있으면 위험도 낮춤
                    if has_positive_context and level in ["medium", "high"]:
                        is_excluded = True

                    if not is_excluded:
                        detected_risks.append(
                            {"keyword": matches[0], "category": level}
                        )

                        if self._is_higher_risk(level, max_risk_level):
                            max_risk_level = level

        # 3. 최종 위험도 조정
        final_risk_level = self._finalize_risk_level(
            max_risk_level, has_positive_context
        )

        return {
            "risk_level": final_risk_level,
            "detected_risks": detected_risks,
            "requires_intervention": final_risk_level in ["critical", "high"],
            "has_positive_context": has_positive_context,
            "risk_message": self._get_risk_message(final_risk_level),
            "timestamp": datetime.now(),
        }

    def _check_positive_context(self, text: str) -> bool:
        """긍정적 맥락 체크"""
        for pattern in self.positive_context:
            if re.search(pattern, text):
                return True
        return False

    def _is_higher_risk(self, level1: str, level2: str) -> bool:
        """위험도 비교"""
        risk_order = {"low": 0, "medium": 1, "high": 2, "critical": 3}
        return risk_order.get(level1, 0) > risk_order.get(level2, 0)

    def _finalize_risk_level(self, max_risk: str, has_positive: bool) -> str:
        """최종 위험도 결정"""
        # 긍정적 맥락이 있으면 위험도 낮춤
        if has_positive and max_risk in ["medium", "high"]:
            return "low"

        # critical은 항상 유지 (생명과 직결)
        if max_risk == "critical":
            return "critical"

        return max_risk

    def _get_risk_message(self, risk_level: str) -> str:
        """위험도별 메시지"""
        messages = {
            "critical": "정말 힘든 상황이시군요. 혼자 견디지 마시고 즉시 도움을 받으세요.",
            "high": "어려운 상황에 계시는 것 같아요. 전문가의 도움을 받아보시는 건 어떨까요?",
            "medium": "스트레스가 있으시군요. 마음을 돌볼 필요가 있을 것 같아요.",
            "low": "괜찮으시다니 다행이에요. 좋은 하루 되세요!",
        }
        return messages.get(risk_level, "안녕하세요.")


class LocationBasedCenterMatcher:
    """위치 기반 센터 매칭 (간단 버전)"""

    def __init__(self, csv_path: str = None):
        if csv_path and os.path.exists(csv_path):
            self.centers = self._load_from_csv(csv_path)
        else:
            self.centers = self._load_default_centers()

        # 주요 도시 좌표
        self.city_coords = {
            "강릉": (37.7519, 128.8761),
            "원주": (37.3422, 127.9202),
            "춘천": (37.8813, 127.7298),
            "창원": (35.2281, 128.6811),
            "김해": (35.2281, 128.8890),
            "진주": (35.1742, 128.0959),
        }

    def _load_from_csv(self, csv_path: str):
        """CSV에서 센터 정보 로드"""
        try:
            df = pd.read_csv(csv_path, encoding="utf-8")
            centers = []

            for _, row in df.iterrows():
                if pd.notna(row.get("센터명")) and pd.notna(row.get("전화번호")):
                    center = SupportCenter(
                        region=str(row.get("시도명", "N/A")),
                        name=str(row.get("센터명")),
                        address=str(row.get("소재지", "N/A")),
                        phone=str(row.get("전화번호")),
                        website=str(row.get("홈페이지", "")),
                        latitude=float(row.get("위도", 0.0)),
                        longitude=float(row.get("경도", 0.0)),
                        center_type=str(row.get("유형", "지원센터")),
                    )
                    if center.latitude != 0.0 and center.longitude != 0.0:
                        centers.append(center)

            print(f"✅ CSV에서 {len(centers)}개 센터 로드")
            return centers

        except Exception as e:
            print(f"❌ CSV 로드 실패: {e}")
            return self._load_default_centers()

    def _load_default_centers(self):
        """기본 센터 데이터"""
        default_data = [
            {
                "region": "강원",
                "name": "강릉중독관리통합지원센터",
                "address": "강릉시 경강로 2279",
                "phone": "033-653-9668",
                "website": "https://gnamc.or.kr/",
                "latitude": 37.7667,
                "longitude": 128.9110,
                "center_type": "중독관리센터",
            },
            {
                "region": "강원",
                "name": "원주시중독관리통합지원센터",
                "address": "원주시 원일로 139",
                "phone": "033-748-5119",
                "website": "http://alja.yonsei.ac.kr",
                "latitude": 37.3515,
                "longitude": 127.9468,
                "center_type": "중독관리센터",
            },
        ]

        return [SupportCenter(**data) for data in default_data]

    def find_nearest_centers(
        self,
        user_coords: Tuple[float, float],
        max_distance: float = 50.0,
        limit: int = 3,
    ):
        """가장 가까운 센터 찾기"""
        if not user_coords:
            return []

        center_distances = []

        for center in self.centers:
            try:
                distance = geodesic(
                    user_coords, (center.latitude, center.longitude)
                ).kilometers

                if distance <= max_distance:
                    center_info = {
                        "name": center.name,
                        "phone": center.phone,
                        "address": center.address,
                        "distance": round(distance, 1),
                        "center_type": center.center_type,
                    }
                    center_distances.append(center_info)
            except:
                continue

        # 거리순 정렬
        center_distances.sort(key=lambda x: x["distance"])
        return center_distances[:limit]

    def get_location_from_text(self, text: str) -> Optional[Tuple[float, float]]:
        """텍스트에서 위치 추출"""
        for city, coords in self.city_coords.items():
            if city in text:
                return coords
        return None


class LocationHandler:
    """위치 정보 처리"""

    def __init__(self, center_matcher):
        self.center_matcher = center_matcher
        self.location_cache = {}

    def process_user_location(
        self,
        user_id: str,
        latitude: float = None,
        longitude: float = None,
        location_text: str = None,
    ):
        """사용자 위치 처리"""

        # GPS 좌표가 있으면 우선 사용
        if latitude and longitude:
            coords = (latitude, longitude)
            return {
                "coordinates": coords,
                "location_source": "gps",
                "location_accuracy": "high",
            }

        # 텍스트에서 위치 추출
        if location_text:
            coords = self.center_matcher.get_location_from_text(location_text)
            if coords:
                return {
                    "coordinates": coords,
                    "location_source": "text",
                    "location_accuracy": "medium",
                }

        # 위치 정보 없음
        return {
            "coordinates": None,
            "location_source": None,
            "location_accuracy": "none",
            "needs_location_request": True,
        }


class YouthLanguageProcessor:
    """청소년 언어 처리"""

    def __init__(self):
        self.youth_slang = {
            "멘붕": "멘탈 붕괴",
            "빡쳐": "화나",
            "레알": "진짜",
            "대박": "놀라워",
        }

    def process_youth_message(self, text: str):
        """청소년 메시지 분석"""
        detected_slang = []

        for slang, meaning in self.youth_slang.items():
            if slang in text:
                detected_slang.append({"slang": slang, "meaning": meaning})

        # 언어 스타일 판단
        if detected_slang or any(char in text for char in ["ㅠㅠ", "ㅜㅜ", "ㅋㅋ"]):
            language_style = "youth_casual"
        else:
            language_style = "formal"

        return {
            "detected_slang": detected_slang,
            "language_style": language_style,
            "emotion_intensity": (
                "high" if "너무" in text or "진짜" in text else "medium"
            ),
        }

## 테스트 해보기

In [13]:
import os
import time
from datetime import datetime
import traceback


class EmotionChatbotTester:
    def __init__(self):
        self.test_results = []
        self.chatbot = None

    def setup_chatbot(self):
        """챗봇 초기화 테스트"""
        print("🔧 챗봇 초기화 중...")
        try:
            self.chatbot = EmotionChatbot(
                model_dir="/Users/hwangeunbi/chatnge_AI/chat/model",
                confidence_threshold=0.6,
                emotion_threshold=0.3,
                enable_safety_features=True,
            )
            print("✅ 챗봇 초기화 성공!")
            return True
        except Exception as e:
            print(f"❌ 챗봇 초기화 실패: {e}")
            traceback.print_exc()
            return False

    def test_basic_conversation(self):
        """기본 대화 테스트"""
        print("\n📝 기본 대화 테스트")
        print("=" * 50)

        test_messages = [
            "안녕하세요!",
            "오늘 날씨가 좋네요",
            "점심 뭐 먹을까요?",
            "주말에 영화 보려고 해요",
        ]

        for msg in test_messages:
            try:
                print(f"\n👤 사용자: {msg}")

                # 스마트 응답 테스트
                if hasattr(self.chatbot, "generate_smart_response"):
                    result = self.chatbot.generate_smart_response(msg)
                    print(f"🤖 AI ({result['model_used']}): {result['ai_response']}")

                    if "emotion_analysis" in result:
                        emotion = result["emotion_analysis"]["final"]["prediction"]
                        confidence = result["emotion_analysis"]["final"]["confidence"]
                        print(f"📊 감정: {emotion} (신뢰도: {confidence:.2f})")
                else:
                    # 기본 응답 테스트
                    response = self.chatbot.generate_response(msg)
                    print(f"🤖 AI: {response}")

                time.sleep(1)  # API 호출 간격

            except Exception as e:
                print(f"❌ 오류: {e}")

    def test_emotion_responses(self):
        """감정 응답 테스트"""
        print("\n💭 감정 응답 테스트")
        print("=" * 50)

        emotion_messages = [
            "오늘 정말 우울해요...",
            "시험에서 떨어져서 너무 속상해요",
            "친구와 싸워서 화가 나요",
            "취업이 안되서 불안해요",
            "연인과 헤어져서 슬퍼요",
        ]

        for msg in emotion_messages:
            try:
                print(f"\n👤 사용자: {msg}")

                if hasattr(self.chatbot, "generate_smart_response"):
                    result = self.chatbot.generate_smart_response(msg)
                    print(f"🤖 AI ({result['model_used']}): {result['ai_response']}")

                    emotion = result["emotion_analysis"]["final"]["prediction"]
                    confidence = result["emotion_analysis"]["final"]["confidence"]
                    print(f"📊 감정: {emotion} (신뢰도: {confidence:.2f})")
                else:
                    response = self.chatbot.generate_response_with_emotion_detection(
                        msg
                    )
                    print(f"🤖 AI: {response}")

                time.sleep(1)

            except Exception as e:
                print(f"❌ 오류: {e}")

    def test_risk_scenarios(self):
        """위험 상황 테스트 (안전 기능 활성화 필요)"""
        print("\n🚨 위험 상황 테스트")
        print("=" * 50)

        risk_messages = [
            "요즘 너무 힘들어서 죽고 싶어요",
            "아무도 날 이해해주지 않아... 혼자 있고 싶어",
            "모든 게 의미없어 보여요",
            "자해하고 싶은 충동이 들어요",
        ]

        for msg in risk_messages:
            try:
                print(f"\n👤 사용자: {msg}")

                if hasattr(self.chatbot, "process_youth_message"):
                    result = self.chatbot.process_youth_message(
                        text=msg,
                        user_id="test_user",
                        latitude=37.5665,  # 서울 좌표
                        longitude=126.9780,
                    )

                    print(f"🤖 AI: {result['ai_response']}")
                    print(
                        f"⚠️ 위험도: {result.get('final_risk_analysis', {}).get('risk_level', 'unknown')}"
                    )
                    print(
                        f"🏥 즉시 도움 필요: {result.get('requires_immediate_help', False)}"
                    )

                    if result.get("nearby_centers"):
                        print(f"🏥 근처 센터: {len(result['nearby_centers'])}개 발견")
                else:
                    response = self.chatbot.generate_response(msg)
                    print(f"🤖 AI: {response}")

                time.sleep(2)  # 위험 상황이므로 더 긴 간격

            except Exception as e:
                print(f"❌ 오류: {e}")

    def test_memory_functionality(self):
        """메모리 기능 테스트"""
        print("\n🧠 메모리 기능 테스트")
        print("=" * 50)

        conversation_flow = [
            "안녕하세요! 제 이름은 김철수예요.",
            "저는 대학생이고 컴퓨터공학과에 다녀요.",
            "제 이름이 뭐였죠?",
            "제가 어떤 전공이라고 했죠?",
        ]

        for msg in conversation_flow:
            try:
                print(f"\n👤 사용자: {msg}")

                response = self.chatbot.generate_response(msg)
                print(f"🤖 AI: {response}")

                # 메모리 상태 확인
                if hasattr(self.chatbot, "get_memory_status"):
                    memory_status = self.chatbot.get_memory_status()
                    print(
                        f"💭 메모리: {memory_status.get('message_count', 0)}개 메시지"
                    )

                time.sleep(1)

            except Exception as e:
                print(f"❌ 오류: {e}")

    def test_model_selection(self):
        """모델 선택 로직 테스트"""
        print("\n🤖 모델 선택 로직 테스트")
        print("=" * 50)

        test_scenarios = [
            ("일반대화", "오늘 점심 뭐 먹을까요?"),
            ("감정상담", "요즘 스트레스가 너무 심해요"),
            ("위험상황", "삶이 너무 힘들어서 포기하고 싶어요"),
        ]

        for scenario_type, msg in test_scenarios:
            try:
                print(f"\n📋 시나리오: {scenario_type}")
                print(f"👤 사용자: {msg}")

                if hasattr(self.chatbot, "_choose_model"):
                    model, emotion_result, risk_analysis = self.chatbot._choose_model(
                        msg
                    )
                    print(f"🎯 선택된 모델: {model}")
                    print(f"📊 감정: {emotion_result['final']['prediction']}")
                    if risk_analysis:
                        print(f"⚠️ 위험도: {risk_analysis.get('risk_level', 'N/A')}")

                time.sleep(1)

            except Exception as e:
                print(f"❌ 오류: {e}")

    def test_threshold_adjustment(self):
        """임계값 조정 테스트"""
        print("\n⚙️ 임계값 조정 테스트")
        print("=" * 50)

        try:
            # 현재 임계값 확인
            if hasattr(self.chatbot, "get_thresholds"):
                current = self.chatbot.get_thresholds()
                print(f"현재 임계값: {current}")

            # 임계값 변경
            if hasattr(self.chatbot, "set_thresholds"):
                self.chatbot.set_thresholds(
                    confidence_threshold=0.8, emotion_threshold=0.4
                )
                print("임계값 변경 완료")

                # 변경된 임계값 확인
                new_thresholds = self.chatbot.get_thresholds()
                print(f"새 임계값: {new_thresholds}")

        except Exception as e:
            print(f"❌ 오류: {e}")

    def test_conversation_summary(self):
        """대화 요약 테스트"""
        print("\n📄 대화 요약 테스트")
        print("=" * 50)

        try:
            # 대화 기록 확인
            if hasattr(self.chatbot, "get_chat_history"):
                history = self.chatbot.get_chat_history()
                print(f"📝 대화 기록: {len(history)}개 메시지")

            # 대화 요약 생성
            if hasattr(self.chatbot, "get_conversation_summary"):
                summary = self.chatbot.get_conversation_summary()
                print(f"📋 대화 요약:\n{summary}")

        except Exception as e:
            print(f"❌ 오류: {e}")

    def run_all_tests(self):
        """모든 테스트 실행"""
        print("🧪 EmotionChatbot 종합 테스트 시작")
        print("=" * 60)

        # 챗봇 초기화
        if not self.setup_chatbot():
            print("❌ 챗봇 초기화 실패로 테스트 중단")
            return

        tests = [
            ("기본 대화", self.test_basic_conversation),
            ("감정 응답", self.test_emotion_responses),
            ("메모리 기능", self.test_memory_functionality),
            ("모델 선택", self.test_model_selection),
            ("임계값 조정", self.test_threshold_adjustment),
            ("대화 요약", self.test_conversation_summary),
            ("위험 상황", self.test_risk_scenarios),  # 마지막에 실행 (민감한 내용)
        ]

        for test_name, test_func in tests:
            try:
                print(f"\n🧪 {test_name} 테스트 시작...")
                test_func()
                print(f"✅ {test_name} 테스트 완료")
            except Exception as e:
                print(f"❌ {test_name} 테스트 실패: {e}")
                traceback.print_exc()

            time.sleep(2)  # 테스트 간 휴식

        print("\n🎉 모든 테스트 완료!")

        # 메모리 정리
        if hasattr(self.chatbot, "clear_memory"):
            self.chatbot.clear_memory()
            print("🧹 메모리 정리 완료")


def main():
    """메인 실행 함수"""
    print("🚀 EmotionChatbot 테스트 프로그램")
    print("=" * 60)

    # API 키 확인
    if (
        not os.getenv("OPENAI_API_KEY")
        or os.getenv("OPENAI_API_KEY") == "your-openai-api-key-here"
    ):
        print("⚠️ OPENAI_API_KEY 환경변수를 설정해주세요!")
        print("export OPENAI_API_KEY='your-actual-api-key'")

    if (
        not os.getenv("ANTHROPIC_API_KEY")
        or os.getenv("ANTHROPIC_API_KEY") == "your-anthropic-api-key-here"
    ):
        print("⚠️ ANTHROPIC_API_KEY 환경변수를 설정해주세요!")
        print("export ANTHROPIC_API_KEY='your-actual-api-key'")

    print("\n실행할 테스트를 선택하세요:")

    print("1. 전체 테스트")
    print("2. 기본 대화만")
    print("3. 감정 응답만")
    print("4. 위험 상황만")
    print("5. 메모리 기능만")

    try:
        choice = input("\n선택 (1-5): ").strip()

        tester = EmotionChatbotTester()

        if choice == "1":
            tester.run_all_tests()
        elif choice == "2":
            if tester.setup_chatbot():
                tester.test_basic_conversation()
        elif choice == "3":
            if tester.setup_chatbot():
                tester.test_emotion_responses()
        elif choice == "4":
            if tester.setup_chatbot():
                tester.test_risk_scenarios()
        elif choice == "5":
            if tester.setup_chatbot():
                tester.test_memory_functionality()
        else:
            print("❌ 잘못된 선택입니다.")

    except KeyboardInterrupt:
        print("\n🛑 테스트가 중단되었습니다.")
    except Exception as e:
        print(f"❌ 테스트 실행 중 오류: {e}")
        traceback.print_exc()


if __name__ == "__main__":
    main()

🚀 EmotionChatbot 테스트 프로그램

실행할 테스트를 선택하세요:
1. 전체 테스트
2. 기본 대화만
3. 감정 응답만
4. 위험 상황만
5. 메모리 기능만
🔧 챗봇 초기화 중...
모델 디렉토리: /Users/hwangeunbi/chatnge_AI/chat/model
디바이스: mps
신뢰도 임계값: 0.6
감정 탐지 임계값: 0.3
1단계 : 파일 확인
✅ level1_best_model.pt 파일이 존재합니다.
✅ level2_best_model.pt 파일이 존재합니다.
✅ level3_best_model.pt 파일이 존재합니다.
✅ level1_label_encoder.pkl 파일이 존재합니다.
✅ level2_label_encoder.pkl 파일이 존재합니다.
✅ level3_label_encoder.pkl 파일이 존재합니다.
2단계: 라벨 인코더 로드
level1_label_encoder.pkl 파일에서 level1_encoder 로드 완료
level2_label_encoder.pkl 파일에서 level2_encoder 로드 완료
level3_label_encoder.pkl 파일에서 level3_encoder 로드 완료
3단계: BERT 모델 로드
BERT 모델과 토크나이저 로드 완료
😊 감정 분류 모델 로드 중...
  🔄 level1_best_model.pt 로드 중...
  ✅ level1_best_model.pt 로드 완료
  🔄 level2_best_model.pt 로드 중...
  ✅ level2_best_model.pt 로드 완료
  🔄 level3_best_model.pt 로드 중...
  ✅ level3_best_model.pt 로드 완료
🌟일반대화용 GPT3.5 초기화완료
🌟감정대화용 GPT4 초기화완료
🌟위험대화용 Cluade 초기화완료
공유 메모리 설정 완료
공유 메모리 및 기본 체인 설정 완료
✅ 청소년 안전 기능 활성화됨
✅ 챗봇 초기화 성공!

🚨 위험 상황 테스트

👤 사용자: 요즘 너무 힘들어서 죽고 싶어요


/var/folders/8n/xfc3hjdn5pq9ch7rw__k4fyh0000gn/T/ipykernel_2862/3263581887.py:410: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.shared_memory = ConversationSummaryBufferMemory(


🤖 AI: 💙 많이 힘드시군요. 혼자가 아니에요.

🚨 **즉시 도움이 필요하다면:**
• 응급상황: 112
• 청소년전화: 1388 (24시간)
• 자살예방상담: 1393
• 위기상담: 1577-0199

혼자가 아니라는 걸 기억해주세요. 💙
⚠️ 위험도: critical
🏥 즉시 도움 필요: True

👤 사용자: 아무도 날 이해해주지 않아... 혼자 있고 싶어


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


🤖 AI: 힘든 마음을 말씀해주셨군요. 그렇게 느끼고 있다니 정말 힘들겠어요. 아무도 당신을 이해해주지 않는다고 느낄 때, 그 외로움과 상처가 얼마나 깊을지 상상할 수 있어요. 지금 어떤 상황이나 감정이 이런 생각을 하게 만들었을까요? 혼자 있고 싶다는 마음이 들 때, 그 공간에서 어떤 것들을 느끼고 싶은지 궁금해요. 당신의 감정을 이렇게 솔직하게 표현해줘서 고마워요.
⚠️ 위험도: low
🏥 즉시 도움 필요: False

👤 사용자: 모든 게 의미없어 보여요
🤖 AI: 힘든 마음을 말씀해주셨군요. 모든 것이 의미 없어 보인다고 느낄 때, 정말 무기력하고 지친 기분일 것 같아요. 그런 감정 속에서 하루하루를 보내는 게 쉽지 않을 거예요. 혹시 최근에 이런 감정을 더 강하게 느끼게 된 어떤 사건이나 변화가 있었을까요? 또는, 예전에는 의미가 있었던 것들이 지금은 왜 그렇게 느껴지는지 생각해본 적 있나요? 당신의 이야기를 듣고 싶어요. 함께 이 감정을 조금씩 풀어가 봐요.
⚠️ 위험도: low
🏥 즉시 도움 필요: False

👤 사용자: 자해하고 싶은 충동이 들어요
🤖 AI: 💙 많이 힘드시군요. 혼자가 아니에요.

🚨 **즉시 도움이 필요하다면:**
• 응급상황: 112
• 청소년전화: 1388 (24시간)
• 자살예방상담: 1393
• 위기상담: 1577-0199

혼자가 아니라는 걸 기억해주세요. 💙
⚠️ 위험도: critical
🏥 즉시 도움 필요: True
